#  EDA & Preprocessing: SAP + Kaggle Bankruptcy Data

**Project:** Integrated Financial Risk Forecasting  
**Datasets:** SAP S/4HANA GBI (6 tables) + Kaggle Bankruptcy (6,819 companies)  
**Goal:** Explore, clean, and prepare data for ML model training

## 1. Import Libraries

In [ ]:
# Core data manipulation
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Preprocessing
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# Suppress warnings for cleaner output
import warnings
warnings.filterwarnings('ignore')

# Display settings
pd.set_option('display.max_columns', 50)
pd.set_option('display.float_format', '{:.2f}'.format)

print(" Libraries loaded!")

## 2. Load Datasets

### 2.1 Kaggle Bankruptcy Data (Labeled - Training)

In [ ]:
# Load Kaggle bankruptcy dataset - 6,819 companies with bankruptcy labels
df_kaggle = pd.read_csv('../data/kaggle/kaggle_company_bankruptcy.csv')

print(f" Kaggle Dataset: {df_kaggle.shape[0]:,} rows × {df_kaggle.shape[1]} columns")
print(f" Target: 'Bankrupt?' → {df_kaggle['Bankrupt?'].value_counts().to_dict()}")

### 2.2 SAP GBI Data (Unlabeled - Application)

In [ ]:
# SAP reading function - skiprows=3 for SE16N header, latin-1 for German chars
def load_sap(table_name):
    return pd.read_csv(f'../data/sap/{table_name}_ALL.txt', 
                       sep='\t', encoding='latin-1', skiprows=3, low_memory=False)

# Load all 6 SAP tables
df_bkpf = load_sap('BKPF')  # Accounting doc headers (38K rows)
df_bseg = load_sap('BSEG')  # Accounting line items (90K rows)
df_bsid = load_sap('BSID')  # Open AR items
df_bsak = load_sap('BSAK')  # Cleared AP items
df_vbak = load_sap('VBAK')  # Sales order headers
df_vbap = load_sap('VBAP')  # Sales order items

# Summary
sap_tables = {'BKPF': df_bkpf, 'BSEG': df_bseg, 'BSID': df_bsid, 
              'BSAK': df_bsak, 'VBAK': df_vbak, 'VBAP': df_vbap}

print(" SAP Tables Loaded:")
for name, df in sap_tables.items():
    print(f"   {name}: {df.shape[0]:,} rows × {df.shape[1]} cols")

---
## 3. Explore Kaggle Dataset

In [ ]:
# Preview first rows - 95 financial ratios + 1 target
df_kaggle.head(3)

In [ ]:
# Class imbalance check - ~3% bankrupt is heavily imbalanced!
fig, ax = plt.subplots(figsize=(6, 4))
df_kaggle['Bankrupt?'].value_counts().plot(kind='bar', color=['green', 'red'], ax=ax)
ax.set_xticklabels(['Healthy (0)', 'Bankrupt (1)'], rotation=0)
ax.set_title('Target Distribution (Imbalanced!)')
ax.set_ylabel('Count')

# Add percentages
for i, v in enumerate(df_kaggle['Bankrupt?'].value_counts()):
    pct = v / len(df_kaggle) * 100
    ax.text(i, v + 50, f'{pct:.1f}%', ha='center', fontweight='bold')

plt.tight_layout()
plt.show()

In [ ]:
# Missing values check - this dataset is clean!
missing = df_kaggle.isnull().sum()
print(f"Missing values: {missing.sum()} (out of {df_kaggle.size:,} cells)")

# Data types - all numeric (good for ML)
print(f"\nData types: {df_kaggle.dtypes.value_counts().to_dict()}")

In [ ]:
# Statistical summary - look for outliers (min/max extremes)
df_kaggle.describe().T.head(10)  # First 10 features

---
## 4. Explore SAP Tables

In [ ]:
# BKPF - Accounting Document Headers (what transactions happened)
print(" BKPF - Document Headers")
print(f"Key columns: {[c for c in df_bkpf.columns if c in ['Company Code', 'Doc. Number', 'Fiscal Year', 'Doc.Date', 'Crcy', 'Doc.Type']]}")
df_bkpf.head(3)

In [ ]:
# VBAK - Sales Order Headers (revenue source)
print(" VBAK - Sales Orders")
print(df_vbak.columns.tolist()[:15])  # First 15 column names
df_vbak.head(3)

In [ ]:
# Quick SAP data quality check
print(" SAP Data Quality Summary:")
for name, df in sap_tables.items():
    missing_pct = df.isnull().sum().sum() / df.size * 100
    print(f"   {name}: {missing_pct:.1f}% missing | {len(df.columns)} columns")

---
## 5. Feature Engineering from SAP → Financial Ratios

Goal: Create features comparable to Kaggle's 95 financial ratios from SAP raw transactions

In [ ]:
# Identify key numeric columns in SAP tables
print(" Key financial columns in SAP:")

# Find columns with 'Net', 'Amount', 'Value' in name
for name, df in sap_tables.items():
    money_cols = [c for c in df.columns if any(x in c.lower() for x in ['net', 'amount', 'value', 'crcy', 'currency'])]
    if money_cols:
        print(f"   {name}: {money_cols[:5]}")  # First 5

In [ ]:
# Calculate basic financial metrics from SAP
# These will be used to create ratios comparable to Kaggle features

# Helper: clean currency columns (German format: 1.234,56 → 1234.56)
def clean_currency(series):
    if series.dtype == 'object':
        return pd.to_numeric(series.str.replace('.', '', regex=False)
                                    .str.replace(',', '.', regex=False), errors='coerce')
    return pd.to_numeric(series, errors='coerce')

# Total Revenue from Sales Orders (VBAK)
if 'Net Value' in df_vbak.columns:
    total_revenue = clean_currency(df_vbak['Net Value']).sum()
    print(f" Total Revenue (VBAK): ${total_revenue:,.2f}")

# Count of transactions
print(f" Total Accounting Docs (BKPF): {len(df_bkpf):,}")
print(f" Total Line Items (BSEG): {len(df_bseg):,}")
print(f" Open AR Items (BSID): {len(df_bsid)}")
print(f" Cleared AP Items (BSAK): {len(df_bsak)}")

---
## 6. Kaggle Data Preprocessing

In [ ]:
# Separate features (X) and target (y)
X = df_kaggle.drop('Bankrupt?', axis=1)
y = df_kaggle['Bankrupt?']

print(f"Features: {X.shape}")
print(f"Target: {y.shape} → {y.value_counts().to_dict()}")

In [ ]:
# Feature scaling - StandardScaler for ML models
# Normalizes features to mean=0, std=1 (important for many algorithms)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled = pd.DataFrame(X_scaled, columns=X.columns)

print(" Features scaled (StandardScaler)")
print(f"   Before: mean={X.iloc[:, 0].mean():.4f}, std={X.iloc[:, 0].std():.4f}")
print(f"   After:  mean={X_scaled.iloc[:, 0].mean():.4f}, std={X_scaled.iloc[:, 0].std():.4f}")

In [ ]:
# Correlation with target - find most predictive features
correlations = df_kaggle.corr()['Bankrupt?'].drop('Bankrupt?').sort_values(key=abs, ascending=False)

print(" Top 10 Features Correlated with Bankruptcy:")
print(correlations.head(10).to_string())

In [ ]:
# Visualize top correlations
fig, ax = plt.subplots(figsize=(10, 6))
top_corr = correlations.head(15)
colors = ['red' if x < 0 else 'green' for x in top_corr.values]
top_corr.plot(kind='barh', color=colors, ax=ax)
ax.set_xlabel('Correlation with Bankruptcy')
ax.set_title('Top 15 Features Most Correlated with Bankruptcy')
ax.axvline(x=0, color='black', linewidth=0.5)
plt.tight_layout()
plt.show()

---
## 7. Data Relationship: SAP ↔ Kaggle Feature Mapping

In [ ]:
# Feature Mapping Table: How SAP data maps to Kaggle features
# This is the KEY bridge between the two datasets

feature_mapping = pd.DataFrame({
    'Kaggle Feature': [
        'Debt ratio %',
        'Current Liability to Assets',
        'Net Income to Total Assets',
        'Total Asset Turnover',
        'Accounts Receivable Turnover',
        'Working Capital to Total Assets',
        'Cash/Total Assets',
        'Revenue Per Person'
    ],
    'SAP Source Tables': [
        'BSEG (Liabilities/Assets)',
        'BSEG (Current Liab/Assets)',
        'BSEG (Net Income/Assets)',
        'VBAK, BSEG (Revenue/Assets)',
        'VBAK, BSID (Revenue/AR)',
        'BSID, BSAK (AR-AP/Assets)',
        'BSEG (Cash accounts)',
        'VBAK (Revenue/headcount)'
    ],
    'SAP Calculation': [
        'Sum(BSEG where HKONT=Liab) / Sum(BSEG where HKONT=Asset)',
        'Current Liab / Total Assets',
        'Net Income / Total Assets from G/L',
        'Total Revenue / Avg Assets',
        'Revenue / Avg AR (from BSID)',
        '(AR - AP) / Total Assets',
        'Cash G/L accounts / Total Assets',
        'Total Revenue / Employee Count'
    ]
})

print(" Feature Mapping: SAP → Kaggle")
feature_mapping

---
## 8. Save Preprocessed Data

In [ ]:
# Create processed data directory
import os
os.makedirs('../data/processed', exist_ok=True)

# Save scaled Kaggle data for model training
kaggle_processed = X_scaled.copy()
kaggle_processed['Bankrupt?'] = y.values
kaggle_processed.to_csv('../data/processed/kaggle_scaled.csv', index=False)

print(f" Saved: data/processed/kaggle_scaled.csv ({len(kaggle_processed):,} rows)")

In [ ]:
# Save top correlated features list for model selection
top_features = correlations.head(20).index.tolist()
pd.DataFrame({'feature': top_features, 
              'correlation': correlations.head(20).values}).to_csv(
    '../data/processed/top_features.csv', index=False)

print(f" Saved: data/processed/top_features.csv (top 20 predictive features)")

---
## 9. Summary & Next Steps

### Data Summary:
| Dataset | Rows | Columns | Purpose |
|---------|------|---------|---------|
| Kaggle Bankruptcy | 6,819 | 96 | Training (labeled) |
| SAP BKPF | 38,179 | 100 | Accounting headers |
| SAP BSEG | 90,476 | 100 | Accounting line items |
| SAP VBAK | 109 | 100 | Sales orders |
| SAP VBAP | 247 | 100 | Sales order items |

### Key Findings:
1. **Class Imbalance**: ~3% bankrupt - will need SMOTE or class weights
2. **Clean Data**: Kaggle has no missing values
3. **Feature Rich**: 95 financial ratios available for training
4. **SAP Ready**: All 6 tables loaded, feature mapping defined

### Next Steps:
1.  Train baseline models (Logistic Regression, Random Forest, XGBoost)
2.  Engineer SAP features to match Kaggle ratios
3.  Apply trained model to SAP data for risk scoring